# LangChain Backend Experiment Number 1

This experiment is to see if we can write a generic chain that takes in a spec and feature list and feedback and tries to improve the feature list and spec list to fit the feedback.

In [108]:
import openai
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

## Sample Project

In [109]:
project_details="""
We're building a comprehensive smart city solution, aiming to enhance the quality of urban life by using digital technology. This solution will have numerous interconnected components and features:

Smart Traffic Management System: Uses AI to analyze real-time traffic data, predict congestion, and control traffic lights to optimize traffic flow. It will also include a predictive maintenance system for traffic infrastructure.

Public Safety System: Incorporates facial recognition technology and predictive policing algorithms to ensure safety. This system will integrate with city surveillance cameras and police databases.

Waste Management System: Implements smart bins equipped with sensors to monitor waste levels and schedule optimal pick-up times.

Smart Energy Grid: Enables real-time monitoring of energy consumption and supply at an individual household level, while also integrating renewable energy sources.

Urban Farming: Incorporates vertical farming technologies, automated irrigation systems, and AI-powered pest prediction and prevention.

E-Government Portal: A central web portal for all government services, integrating multiple departmental systems and databases, complete with secure identity verification and multi-language support.

Public Transportation System: A fully automated scheduling and ticketing system for all city public transportation, including buses, subways, and bike rentals. It will use AI to optimize routes and schedules based on real-time data.

Environment Monitoring: Implementing IoT sensors across the city to monitor air and water quality in real time.
"""
project_technologies="""
The project will be developed using a range of technologies including, but not limited to, Python, TensorFlow, React.js, Node.js, PostgreSQL, Docker, Kubernetes, AWS, IoT protocols, and various data science and AI/ML libraries.

This system will have to comply with stringent regulations and standards for data security and privacy, including GDPR and ISO 27001. It will also require robust scalability, fault tolerance, and multi-region availability to cater to the entire city's population.

Additionally, the system will need to be user-friendly, accessible, and offer multi-language support to cater to the diverse user base in the city.
"""

### Setting Up Chain No. 1: Feature Retrieval

In [110]:

llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")
prompt = PromptTemplate(
    input_variables=["project_details", "project_technologies"],
    template="""
    Given the following project description and tech stack, identify and elaborate on the key backend features that would be necessary for development. The backend features should not involve any frontend components or styling. The backend features should be described in terms of capabilities.
    This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    Project description: {project_details}
    Technologies: {project_technologies}
    """
)

chain = LLMChain(llm=llm, prompt=prompt)
backend_features = chain.run({
    'project_details': project_details,
    'project_technologies': project_technologies
})
print(backend_features)

MVP Backend Features:
1. Smart Traffic Management System:
- Real-time traffic data analysis: Develop algorithms to analyze real-time traffic data and identify congestion patterns.
- Traffic light control: Implement a system to control traffic lights based on the analyzed data to optimize traffic flow.
- Predictive maintenance system: Create a system to monitor the condition of traffic infrastructure and predict maintenance needs.

2. Public Safety System:
- Facial recognition integration: Integrate facial recognition technology to identify individuals from surveillance cameras.
- Predictive policing algorithms: Develop algorithms to analyze crime data and predict potential crime hotspots.
- Integration with police databases: Connect the system with police databases to access relevant information about individuals.

3. Waste Management System:
- Sensor integration: Develop a system to integrate sensors with smart bins to monitor waste levels.
- Optimal pick-up scheduling: Implement algo

### Setting Up Chain Two:  Feature Extraction

In [111]:
specification_prompt = PromptTemplate(
    input_variables=["backend_features", "project_technologies"],
    template="""
    Given the extracted backend features and the specified skills/technologies, create a detailed technical specification. 
    This specification should include the technologies to be used, the architecture, the different routes/endpoints, their inputs and outputs, and any potential hardware and startup costs.
    However, they should be split into two categories: MVP and Additional features. The MVP should be the minimum features necessary to have a working prototype.
    You should ignore the technologies for the frontend and focus on the backend.
    Please also mention any other technical considerations.
    
    Backend Features: {backend_features}
    
    Project Technologies: {project_technologies}
    """
)

specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
specification = specification_chain.run({
    'backend_features': backend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

MVP Backend Features:
1. Smart Traffic Management System:
- Real-time traffic data analysis:
  - Technology: Python, TensorFlow
  - Architecture: Develop algorithms to analyze real-time traffic data and identify congestion patterns.
  - Inputs: Real-time traffic data from sensors and cameras.
  - Outputs: Congestion patterns and traffic flow optimization recommendations.

- Traffic light control:
  - Technology: Node.js, IoT protocols
  - Architecture: Implement a system to control traffic lights based on the analyzed data to optimize traffic flow.
  - Inputs: Congestion patterns and traffic flow optimization recommendations.
  - Outputs: Control signals for traffic lights.

- Predictive maintenance system:
  - Technology: Python, TensorFlow
  - Architecture: Create a system to monitor the condition of traffic infrastructure and predict maintenance needs.
  - Inputs: Sensor data from traffic infrastructure.
  - Outputs: Maintenance predictions and alerts.

2. 

### Setting Up Chain Three: AI Approval

In [112]:
approval_prompt = PromptTemplate(
    input_variables=["technical_specification", "aspect", "group_size", "group_experience"],
    template="""
    Given the developed technical specification, conduct a thorough review of the MVP Features only for any inconsistencies or issues. 
    Also, evaluate whether the MVP Features, can be realistically completed within the two day hackathon for {group_size} people, considering the complexity and the technology stack required.
    
    The MVP Features are specifically listed under the heading 'MVP Features'. 
    Please completely disregard any features or sections listed under 'Additional Features' or any similar headers.
    This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects.

    Answer this question: Can the MVP Features be realistically completed within the two day hackathon for {group_size} people with this skill level: {group_experience}?
    Output only a json with keys 'approval' and 'comments'. 
    If yes, the value of 'approval' should be '1' and the value of 'comments' should be an empty string
    If not, the value of 'approval' should be '0' and the value of 'comments' should be a string with the issues and inconsistencies listed.

    Technical Specification: {technical_specification}
    
    Output only a parsable one line json with keys 'approval' and 'comments'. 
    """
)
advanced_llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-4")
approval_chain = LLMChain(llm=advanced_llm, prompt=approval_prompt)
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'backend',
    'group_size': '4',
    'group_experience': "experienced"
})
print(approval)

{"approval": "0", "comments": "The MVP features listed are highly complex and require a significant amount of time and resources to implement. Even with an experienced team of 4 people, it is unrealistic to complete these features within a two-day hackathon. The features involve complex algorithms, integration with various data sources and systems, and the use of advanced technologies like TensorFlow for machine learning. Additionally, the features span across multiple domains like traffic management, public safety, waste management, and energy which adds to the complexity. It would be more feasible to focus on a subset of these features for the hackathon."}


### Setting Up Chain 4: AI Revision

In [113]:
revision_prompt = PromptTemplate(
    input_variables=["original_features", "comments"],
    template="""
    Given the original features and the comments provided after the feasibility evaluation, modify the original features to make them simpler and more feasible for a two-day hackathon.
    The aim is to reduce the complexity and scope of the original features while preserving the core functionality of the system. 
    The comments provide specific areas of concern that should be addressed in the modified features.

    Original Features: {original_features}
    Comments: {comments}

    Based on this, please provide a simplified version of the original features that addresses the concerns mentioned in the comments.
    """
)

approval_object = json.loads(approval)
revision_chain = LLMChain(llm=llm, prompt=revision_prompt)
backend_features = revision_chain.run({
    'original_features': backend_features,
    'comments': approval_object['comments']
})

print(backend_features)


Modified Features for a Two-Day Hackathon:

1. Smart Traffic Management System:
- Real-time traffic data analysis: Develop a simplified algorithm to analyze real-time traffic data and identify congestion patterns.
- Traffic light control: Implement a basic system to control traffic lights based on the analyzed data to optimize traffic flow.

2. Public Safety System:
- Facial recognition integration: Integrate a basic facial recognition technology to identify individuals from surveillance cameras.
- Predictive policing algorithms: Develop a simplified algorithm to analyze crime data and identify potential crime hotspots.

3. Waste Management System:
- Sensor integration: Develop a basic system to integrate sensors with smart bins to monitor waste levels.
- Optimal pick-up scheduling: Implement a simplified algorithm to schedule waste pick-ups based on the monitored waste levels.

4. Smart Energy Grid:
- Real-time energy monitoring: Create a basic system to monitor energy consumption and

Now lets regenerate the specifications

In [114]:
specification = specification_chain.run({
    'backend_features': backend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

Minimum Viable Product (MVP):

1. Smart Traffic Management System:
- Real-time traffic data analysis:
  - Technology: Python, TensorFlow, data science libraries
  - Inputs: Real-time traffic data
  - Outputs: Congestion patterns analysis
- Traffic light control:
  - Technology: Python, Node.js, IoT protocols
  - Inputs: Analyzed traffic data
  - Outputs: Traffic light control signals

2. Public Safety System:
- Facial recognition integration:
  - Technology: Python, TensorFlow, facial recognition libraries
  - Inputs: Surveillance camera footage
  - Outputs: Identified individuals
- Predictive policing algorithms:
  - Technology: Python, data science libraries
  - Inputs: Crime data
  - Outputs: Potential crime hotspots analysis

3. Waste Management System:
- Sensor integration:
  - Technology: Python, IoT protocols
  - Inputs: Sensor data from smart bins
  - Outputs: Waste level monitoring
- Optimal pick-up scheduling:
  - Technology: Python, scheduling algor

Finally lets do another feasibility test

In [115]:
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'backend',
    'group_size': '4',
    'group_experience': "experienced"
})
print(approval)

{"approval": "0", "comments": "The MVP features listed in the technical specification are quite complex and require a wide range of skills and technologies. Even with an experienced team of 4 people, it is unlikely that all these features can be realistically completed within a two day hackathon. The features involve complex tasks such as real-time data analysis, facial recognition, predictive algorithms, IoT integration, and energy grid management, all of which require significant time for development, testing, and debugging. Additionally, the technologies required (Python, TensorFlow, Node.js, IoT protocols, data science libraries) are diverse and each require specialized knowledge. It would be more realistic to focus on a subset of these features for the hackathon."}


As you can see, this new mvp is much more feasible than the previous one, but yet still not the best. We can keep on iterating on this until we get a good enough mvp.
But Now, we will define a reusable function instead of calling each chain one by one.

In [116]:
def revision(old_features, specification, approval):
    approval_object = json.loads(approval)
    if approval_object['approval'] == '1':
        return specification
    else:
        backend_features = revision_chain.run({
            'original_features': old_features,
            'comments': approval_object['comments']
        })
        
        specification = specification_chain.run({
            'backend_features': backend_features,
            'project_technologies': project_technologies
        })
        
        approval = approval_chain.run({
            'technical_specification': specification,
            'aspect': 'backend',
            'group_size': '4',
            'group_experience': "experienced"
        })
        
        return approval, specification, backend_features
        
        
approval, specification, backend_features = revision(backend_features, specification, approval)
print(approval)

{"approval": "0", "comments": "The MVP features are complex and require a wide range of technologies. The Smart Traffic Management System and Public Safety System both require TensorFlow, which is a machine learning library, indicating that these systems involve machine learning models. Developing and training these models within a two-day hackathon is unrealistic, even for experienced developers. Additionally, the Waste Management System involves an algorithm for optimal pick-up scheduling, which could be complex depending on the factors considered. Given these complexities and the breadth of technologies required, it is unlikely that these features can be realistically completed within the two-day hackathon by four people."}


The comment is less harsh now, but still not perfect. We can keep on iterating on this until we get a good enough mvp. But lets take a look the backend features again

In [117]:
print(backend_features)

Modified Features for a Two-Day Hackathon:

1. Smart Traffic Management System:
- Real-time traffic data analysis: Develop a simplified algorithm to analyze real-time traffic data and identify congestion patterns.
- Traffic light control: Implement a basic system to control traffic lights based on the analyzed data to optimize traffic flow.

2. Public Safety System:
- Facial recognition integration: Integrate a basic facial recognition technology to identify individuals from surveillance cameras.

3. Waste Management System:
- Sensor integration: Develop a basic system to integrate sensors with smart bins to monitor waste levels.
- Optimal pick-up scheduling: Implement a simplified algorithm to schedule waste pick-ups based on the monitored waste levels.

Additional Backend Features:
1. Urban Farming:
- Vertical farming technology integration: Implement a basic system to support vertical farming technologies.

2. E-Government Portal:
- Integration with departmental systems and database

Lets see if revising 5 times would make a difference

In [118]:
print(approval)

{"approval": "0", "comments": "The MVP features are complex and require a wide range of technologies. The Smart Traffic Management System and Public Safety System both require TensorFlow, which is a machine learning library, indicating that these systems involve machine learning models. Developing and training these models within a two-day hackathon is unrealistic, even for experienced developers. Additionally, the Waste Management System involves an algorithm for optimal pick-up scheduling, which could be complex depending on the factors considered. Given these complexities and the breadth of technologies required, it is unlikely that these features can be realistically completed within the two-day hackathon by four people."}


In [119]:
for i in range(5):
    approval, specification, backend_features = revision(backend_features, specification, approval)
    print(approval)
    print(backend_features)
    print("=========================================")
    if(json.loads(approval)['approval'] == '1'):
        break

{"approval": "0", "comments": "The MVP features listed are quite complex and require a wide range of technologies and skills. Real-time data analysis, facial recognition, and IoT integration are all advanced features that require significant time and expertise to implement. Even with an experienced team of 4 people, it would be challenging to complete all these features within a two-day hackathon. It would be more realistic to focus on one or two of these features for the hackathon."}
Modified Features for a Two-Day Hackathon:

1. Smart Traffic Management System:
- Real-time traffic data analysis: Develop a simplified algorithm to analyze real-time traffic data and identify congestion patterns.
- Traffic light control: Implement a basic system to control traffic lights based on the analyzed data to optimize traffic flow.

2. Public Safety System:
- Facial recognition integration: Integrate a basic facial recognition technology to identify individuals from surveillance cameras.

3. Wast